In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopy.distance
import geopy.point


In [80]:
df_skiers = pd.read_csv("../data/filtered/Dati_filtrati_all.csv")
df_skiers = df_skiers[df_skiers["DNF"] == False]

In [81]:
vel_df = pd.DataFrame(columns=["Athlete", "Day", "0 - 60", "60 - 65", "65 - 70", "70 - 75", "75 - 80", "80 - 85", "85+"])

for day in ["G1", "P1", "P2"]:
    df_skiers_day = df_skiers[df_skiers["Day"] == day]
    n_intervals = 1 + 5 + 1
    global_intervals = [0] * n_intervals

    for athlete, skier_file, startTime, duration in zip(df_skiers_day["Atleta"], df_skiers_day["File_gps"], df_skiers_day["StartTime"], df_skiers_day["Time"]):
        df_skier = pd.read_csv(skier_file)
        df_skier_started = df_skier[df_skier["Timestamp"] >= startTime]
        df_skier_run = df_skier_started[df_skier_started["Timestamp"] <= startTime + duration]
        df_skier_run = df_skier_run.sort_values('Timestamp', ascending=False)
        intervals = [0] * n_intervals

        for velocity, long1, lat1, alt1, long2, lat2, alt2 in zip(df_skier_run["GNSS_SpeedOverGround"][:-1], df_skier_run["GNSS_Longitude"][:-1], df_skier_run["GNSS_Latitude"][:-1], df_skier_run["GNSS_Altitude"][:-1],df_skier_run["GNSS_Longitude"][1:], df_skier_run["GNSS_Latitude"][1:],df_skier_run["GNSS_Altitude"][1:]):

            distance2d = geopy.distance.geodesic((long1, lat1), (long2, lat2)).m
            distance = np.sqrt(distance2d**2 + (alt1 - alt2)**2)

            if velocity <= 60:
                intervals[0] += distance
                global_intervals[0] += distance
            elif velocity >= 85:
                intervals[7] += distance
                global_intervals[7] += distance
            else:
                intervals[int(np.floor((velocity - 60) / 5)) + 1] += distance
                global_intervals[int(np.floor((velocity - 60) / 5)) + 1] += distance        
        plt.bar(range(1, 8), intervals)
        plt.xlabel("Velocity (km/h)")
        plt.ylabel("Distance travelled (m)")
        plt.xticks(range(1, 8), ["0 - 60", "60 - 65", "65 - 70", "70 - 75", "75 - 80", "80 - 85", "85+"])
        plt.title(f"Travelled distance in different velocity ranges of skier {athlete} on day {day}")
        plt.savefig(f"../figures/Travelled_distance_in_different_velocity_ranges_of_skier_{athlete}_on_day_{day}.png")
        plt.close()

        intervals.insert(0, day)
        intervals.insert(0, athlete)
        vel_df.loc[len(vel_df)] = intervals


    plt.bar(range(1, 8), global_intervals)
    plt.xlabel("Velocity (km/h)")
    plt.ylabel("Distance travelled (m)")
    plt.xticks(range(1, 8), ["0 - 60", "60 - 65", "65 - 70", "70 - 75", "75 - 80", "80 - 85", "85+"])
    plt.title(f"Travelled distance in different velocity ranges of skiers on day {day}")
    plt.savefig(f"../figures/Travelled_distance_in_different_velocity_ranges_of_skiers_on_day_{day}.png")
    plt.close()
vel_df.to_csv("../data/filtered/velocity_ranges.csv", index=False)